In [ ]:
! sudo apt-get -y install wget git
! wget --no-check-certificate -P . https://dumps.wikimedia.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2
! bzip2 -d fawiki-latest-pages-articles-multistream.xml.bz2
! pip install wikiextractor
! python -m wikiextractor.WikiExtractor --json fawiki-latest-pages-articles-multistream.xml




git is already the newest version (1:2.25.1-1ubuntu3.6).
wget is already the newest version (1.20.3-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--2022-12-01 03:37:23--  https://dumps.wikimedia.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1119345079 (1.0G) [application/octet-stream]
Saving to: ‘./fawiki-latest-pages-articles-multistream.xml.bz2’

atest-pages-article  22%[===>                ] 236.91M  3.91

*** WARNING: max output size exceeded, skipping output. ***

fawiki-latest-pages 100%[===================>]   1.04G  3.96MB/s    in 4m 28s  

2022-12-01 03:41:52 (3.98 MB/s) - ‘./fawiki-latest-pages-articles-multistream.xml.bz2’ saved [1119345079/1119345079]

     |████████████████████████

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MDA_2021").master("local[*]").getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/02 12:38:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
articles_rdd = sc.textFile("text/*/*")
articles_rdd.take(1)

['{"id": "2", "revid": "23528", "url": "https://fa.wikipedia.org/wiki?curid=2", "title": "\\u0635\\u0641\\u062d\\u0647\\u0654 \\u0627\\u0635\\u0644\\u06cc", "text": "&lt;templatestyles src=\\"\\u0635\\u0641\\u062d\\u0647 \\u0627\\u0635\\u0644\\u06cc/styles.css\\"/&gt;\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n\\u0627\\u0645\\u0631\\u0648\\u0632: \\u060c \\u0645\\u06cc\\u0644\\u0627\\u062f\\u06cc \\u0628\\u0631\\u0627\\u0628\\u0631 \\u0647\\u062c\\u0631\\u06cc \\u062e\\u0648\\u0631\\u0634\\u06cc\\u062f\\u06cc \\u0648 (UTC)\\n \\n \\n \\n \\n \' \\u2013 \'\\n \\n \\n \\n \\n \\n \\n __NOEDITSECTION__"}']

In [3]:
import json
import time
# Jsonify
start = time.time()
json_rdd = articles_rdd.map(lambda x : json.loads(x))
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 0.000min


In [4]:
def cleansing(x):
    dirty_words=['\u200c','\n','(',')','-','.',',',"<<",">>",'"',":","»","«","،","[","]","}","{","?","!","@","#","$","&","*","_",'"',">","<","؛","/",'-','%','+','↓','|','\u200e','\u202d','\u202c','=','ـ','\uf0fc','\u200f','\u200d','%',';','°','//','\u200b','×','.','؟','.','–','۔','\\','^']
    title = x['title']
    text = x['text']
    for word in dirty_words:
        title = title.replace(word," ")
    for word in dirty_words:
        text = text.replace(word, " ")
    title = title.replace("  ", " ")
    text = text.replace("  ", " ")
    x.update({"title" : title})
    x.update({"text" : text})
    return x

start = time.time()
cleansed_articles_rdd = json_rdd.map(lambda x : cleansing(x)) #cleansed text
# cached_cleansed_articles = cleansed_articles_rdd.cache()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 0.000min


In [5]:
words_rdd = cleansed_articles_rdd.flatMap(lambda x : (x['title'], x['text'])).flatMap(lambda x : x.split(" "))

In [6]:
words_rdd.count()

134550101

In [7]:
start = time.time()
words_count_rdd = words_rdd.map(lambda x : (x,1))
# lst = words_count_rdd.collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 0.000min


In [8]:
start = time.time()
words_count = words_count_rdd.reduceByKey(lambda x, y : x+y)
# lst = words_count_rdd.collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 0.001min


In [9]:
start = time.time()
top_100 = words_count.map(lambda x : (x[1], x[0])).sortByKey(False)
lst = top_100.collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")
lst = lst[:100]
lst

22/12/02 13:09:19 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1390166 ms exceeds timeout 120000 ms
22/12/02 13:09:19 WARN SparkContext: Killing executors is not supported by current scheduler.


Time taken: 35.263min


[(5120192, 'در'),
 (4819648, ''),
 (4590305, 'و'),
 (3433408, 'به'),
 (2987074, 'از'),
 (2674944, 'است'),
 (2221050, 'که'),
 (2070077, 'می'),
 (1576166, 'این'),
 (1430712, 'را'),
 (1399740, 'های'),
 (1169385, 'با'),
 (992247, 'یک'),
 (874752, 'آن'),
 (869352, 'شده'),
 (835379, 'سال'),
 (736676, 'ها'),
 (698692, 'شد'),
 (630279, 'بود'),
 (623113, 'برای'),
 (546070, 'کرد'),
 (519567, 'او'),
 (494600, 'شود'),
 (439410, 'ای'),
 (437361, 'دارد'),
 (369873, 'خود'),
 (369431, 'بر'),
 (361920, 'تا'),
 (350439, 'ایران'),
 (347212, 'یا'),
 (333273, 'کند'),
 (319710, 'واقع'),
 (300621, 'وی'),
 (292765, 'عنوان'),
 (290136, 'فوتبال'),
 (285008, 'نام'),
 (276296, 'جمعیت'),
 (273236, 'قرار'),
 (260373, 'نیز'),
 (255018, 'باشد'),
 (246517, 'فیلم'),
 (239946, 'کرده'),
 (239914, 'دو'),
 (237577, 'نفر'),
 (231783, 'پس'),
 (224977, 'بازی'),
 (218840, 'باشگاه'),
 (212176, 'شهر'),
 (211958, 'شهرستان'),
 (207397, 'استفاده'),
 (207011, 'بوده'),
 (206903, 'یکی'),
 (205564, 'توسط'),
 (201164, 'اهل'),
 (196910, 

In [10]:
# TODO : Check for stopwords 
# Just delete the first 1000
# Create TF-IDF
stop_words = ['در','و','به','از','است','که','می','این','','را','های','با','آن','شده','سال','ها','شد','بود','برای','کرد','او','شود','ای','دارد','بر','خود','تا','یا','کند','وی','نیز','باشد','کرده','پس','یکی','بوده','دیگر','دیگر','همچنین','توسط','هم','دیگر','اند','اما','هایی','کنند','هر','مورد','وجود','هستند','داد','داشته','داشت','روی','زیر','دهد','شوند','کردند']
stopwords = ['در','و','به','از','که','','را','این','با','است','آن','برای','او','تا','یا','بر','شده','شدهاست.','شد.','نیز','بود','کرد.','پس','بود.','آنها','دارد','همچنین',')','دارد.','کرد','اما','میتوان','(؛','هر','هم','میشود.','میشود','می','است،','دارای']
new_stopword = ['شدهاست','قرار','میکند','بین','بودهاست','کردهاست','بعد','مانند','بودند','اس','میکنند','برخی','میدهد','گرفت','همراه','میشوند','بسیاری','شدن','دارند','کردن','گرفته','شدند','حدود','میتواند','ولی','شکل','بیش','مربوط','همین','اینکه','باعث','سپس','براساس','یافت','همان','میشد','قابل','اگر','میکرد','گفته','همه','دیگری','نمود','معمولاً','خواهد','سوی','طی','آنجا','یعنی','رسید','جای','دربارهٔ','کنار','حتی','سایر','شدهاند','چندین','چنین','میگیرد','رفت', 'پی','هنگام','مقابل','بی','بودن','حالی','آمد','گردید','فقط','آنان','داشتند','اغلب','رسیده','کلی','چه','باز','مثال','علاوه','دادن','میکردند','رو','باشند','میرود','منجر','ندارد','کردهاند','دیده','میدهند','داخل','هاي','میرسد', 'هنوز', 'وقتی', 'ان', 'مثل', 'اوایل', 'هنگامی', 'باقی', 'میگوید', 'دادند', 'نامیده', 'آمده', 'بعضی', 'یکدیگر', 'موجب', 'بعدها', 'پيش', 'تر', 'میآید', 'کمی', 'گرفتن', 'البته', 'البته', 'بلکه', 'گاهی', 'کسی', 'درون', 'خودش', 'نامهای','یافته', 'جایی', 'مدتی', 'اي', 'میگردد','کاملاً', 'سرانجام', 'تمامی', 'اکنون', 'اگرچه', 'تعدادی', 'چیزی']
stp = stop_words + stopwords + new_stopword


In [11]:
def remove_words(x, words):
    title = x['title'].split(" ")
    lst = []
    for w in title:
        if w not in words:
            lst.append(w)
    while '' in lst:
        lst.remove('')

    str_1 = " ".join(lst)
    x.update({'title':str_1})
    text = x['text'].split(" ")
    lst = []
    for w in text:
        if w not in words:
            lst.append(w)
    while '' in lst:
        lst.remove('')

    str_2 = " ".join(lst)
    x.update({'text': str_2})
    return x

start = time.time()
articles_without_stopwords_rdd = cleansed_articles_rdd.map(lambda x : remove_words(x, stp))
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 0.000min


In [12]:
start = time.time()
top_100 = articles_without_stopwords_rdd.flatMap(lambda x : (x['title'], x['text'])).flatMap(lambda x : x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda x, y: x + y).map(lambda x : (x[1], x[0])).sortByKey(False)
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")


Time taken: 8.476min


In [13]:
start = time.time()
MIN_COUNT = 20
uncommon_words = top_100.filter(lambda x : x[0] < MIN_COUNT).map(lambda x : x[1]).collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 2.992min


: 

In [ ]:
start = time.time()
MIN_COUNT = 20
common_words = top_100.filter(lambda x : x[0] > MIN_COUNT).map(lambda x : x[1]).collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 0.624min


In [ ]:

hash_list = {}
            
for i in range(401):
    hash_list[str(i)] = []
    
    
str((ord(i[0]) + ord(i[-1])) * len(i) % 401)

for i in common_words:
    if len(i) != 0:
        hash_list[str((ord(i[0])+ord(i[-1]))*len(i)%401)].append(i)

def remove_uncommon(x, dic):    
    title = x['title'].split(" ")
    lst = []
    for w in title:
        if len(w) > 0:
            hashed_w = str((ord(w[0])+ord(w[-1]))*len(w)%401)
            if w in dic[hashed_w]:
                lst.append(w)
    str_1 = " ".join(lst)
    x.update({'title': str_1})
    text = x['text'].split(" ")
    lst = []
    for w in text:
        if len(w) > 0:
            hashed_w = str((ord(w[0])+ord(w[-1]))*len(w)%401)
            if w in dic[hashed_w]:
                lst.append(w)
    str_2 = " ".join(lst)
    x.update({'text': str_2})
    return x


start = time.time()
articles_cleaned_rdd = articles_without_stopwords_rdd.map(lambda x : remove_uncommon(x, hash_list))
awsr = articles_cleaned_rdd.collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")



Time taken: 1.254min


* How many unique 3-Letter words remain after the cleaning procedure?

In [ ]:
unique_3_letter = articles_cleaned_rdd.flatMap(lambda x : (x['title'], x['text'])).flatMap(lambda x : x.split(" ")).filter(lambda x : len(x) == 3).distinct().count()
unique_3_letter

Out[123]: 8289

* What are the top 20 most common English trigrams in the corpus?

In [ ]:
def trigram(text, size=3):
    tokens = text.split()
    return [' '.join(tokens[i:i+size])
                     for i in range(len(tokens) - size + 1)]
    
def finding_english(x):
    if len(x) > 2:
        if ('a' <= x[0] <= "z" or 'A' <= x[0] <='Z') and ('a' <= x[1] <= "z" or 'A' <= x[1] <='Z') and ('a' <= x[1] <= "z" or 'A' <= x[1] <='Z'):
            return x
    
# articles_cleaned_rdd.flatMap(lambda x : (x['title'], x['text'])).map(lambda x : trigram(x)).filter(lambda x : len(x) >= 3).filter(lambda x : ('a' <= x[0] <= "z" or 'A' <= x[0] <='Z') and ('a' <= x[1] <= "z" or 'A' <= x[1] <='Z') and ('a' <= x[2] <= "z" or 'A' <= x[2] <='Z')).take(4)
# articles_cleaned_rdd.flatMap(lambda x : (x['title'], x['text'])).map(lambda x : trigram(x)).collect
articles_cleaned_rdd.flatMap(lambda x : (x['title'], x['text'])).flatMap(lambda x : trigram(x)).collect()


Out[73]: ['رادیو سامانه صدا',
 'سامانه صدا پیام',
 'صدا پیام حالت',
 'پیام حالت یک',
 'حالت یک سیگنال',
 'یک سیگنال وسیله',
 'سیگنال وسیله امواج',
 'وسیله امواج رادیویی',
 'امواج رادیویی منتقل',
 'رادیویی منتقل ارسال',
 'منتقل ارسال دریافت',
 'ارسال دریافت ارتباط',
 'دریافت ارتباط رادیویی',
 'ارتباط رادیویی ارسال',
 'رادیویی ارسال سیگنال',
 'ارسال سیگنال استفاده',
 'سیگنال استفاده امواج',
 'استفاده امواج الکترومغناطیسی',
 'امواج الکترومغناطیسی فرکانس',
 'الکترومغناطیسی فرکانس رادیویی',
 'فرکانس رادیویی امواج',
 'رادیویی امواج الکترومغناطیسی',
 'امواج الکترومغناطیسی توانند',
 'الکترومغناطیسی توانند هوا',
 'توانند هوا اطلاعات',
 'هوا اطلاعات صدا',
 'اطلاعات صدا تصویر',
 'صدا تصویر پیام',
 'تصویر پیام استفاده',
 'پیام استفاده روش',
 'استفاده روش معین',
 'روش معین تغییر',
 'معین تغییر مشخصه',
 'تغییر مشخصه امواج',
 'مشخصه امواج منتشر',
 'امواج منتشر دامنه',
 'منتشر دامنه فرکانس',
 'دامنه فرکانس فاز',
 'فرکانس فاز پهنای',
 'فاز پهنای منتقل',
 'پهنای منتقل امواج',
 'منتقل امواج الکترومغناطیس

In [ ]:
articles_cleaned_rdd.flatMap(lambda x : (x['title'], x['text'])).flatMap(lambda x : x.split(" ")).filter(lambda x : len(x) > 0).filter(lambda x : 'a' <= x[0] <= "z" or 'A' <= x[0] <='Z').map(lambda x : (x,1)).reduceByKey(lambda x, y: x+y).map(lambda x : (x[1], x[0])).sortByKey(False).collect()

Out[74]: [(625, 'formula'),
 (222, 'lt'),
 (220, 'gt'),
 (169, 'of'),
 (98, 'mTOR'),
 (96, 'the'),
 (91, 'A'),
 (79, 'x'),
 (73, 'The'),
 (73, 'amp'),
 (70, 'bdi'),
 (68, 'de'),
 (67, 'and'),
 (65, 'http'),
 (59, 'County'),
 (59, 'B'),
 (57, 'a'),
 (56, 'C'),
 (55, 'org'),
 (49, 'Wikipedia'),
 (47, 'g'),
 (46, 'in'),
 (46, 'S'),
 (43, 'en'),
 (41, 'https'),
 (40, 'com'),
 (40, 'to'),
 (39, 'www'),
 (39, 'wikipedia'),
 (38, 'Township'),
 (38, 'p'),
 (37, 'District'),
 (37, 'II'),
 (36, 'team'),
 (35, 'national'),
 (34, 'football'),
 (34, 'm'),
 (33, 'mol'),
 (33, 'I'),
 (33, 'D'),
 (32, 'UEFI'),
 (32, 'P'),
 (32, 'under'),
 (30, 'if'),
 (29, 'q'),
 (29, 'EFI'),
 (28, 'K'),
 (28, 'Canton'),
 (28, 'O'),
 (28, 'E'),
 (27, 'w'),
 (26, 'title'),
 (26, 'php'),
 (25, 'SKP'),
 (25, 'Free'),
 (25, 'SLS'),
 (25, 'Wget'),
 (24, 'III'),
 (24, 'contributors'),
 (24, 'oldid'),
 (24, 'Encyclopedia'),
 (24, 'index'),
 (23, 'int'),
 (22, 'V'),
 (22, 'H'),
 (22, 'b'),
 (21, 'R'),
 (21, 'T')]

In [ ]:
start = time.time()
doc_map = articles_cleaned_rdd.flatMap(lambda x: [((x['id'],i),1) for i in x['text'].split()])
doc_map.collect()
end = time.time()
print(f"Time taken: {(end-start)*10**3/60/1000:.03f}min")

Time taken: 1.280min


In [ ]:
aggregated_doc = doc_map.reduceByKey(lambda x, y: x+y).sortByKey(False)
ag_cache = aggregated_doc.cache()
ag = aggregated_doc.collect()
ag

Out[26]: [(('988788', '۵'), 1),
 (('988788', '۴'), 1),
 (('988788', '۳'), 1),
 (('988788', 'یک'), 1),
 (('988788', 'کم'), 1),
 (('988788', 'پاب'), 1),
 (('988788', 'شیمیایی'), 1),
 (('988788', 'شناسه'), 1),
 (('988788', 'تری'), 1),
 (('988788', 'ترکیب'), 1),
 (('988788', 'اسید'), 1),
 (('9881', 'کار'), 1),
 (('9881', 'چند'), 1),
 (('9881', 'واژه'), 1),
 (('9881', 'معنی'), 1),
 (('9881', 'رود'), 1),
 (('9881', 'خاور'), 1),
 (('975009', '۲'), 1),
 (('975009', '۱۳۸۸'), 1),
 (('975009', 'یک'), 1),
 (('975009', 'گذار'), 1),
 (('975009', 'کشد'), 1),
 (('975009', 'کتابی'), 1),
 (('975009', 'کتاب'), 1),
 (('975009', 'چاپ'), 1),
 (('975009', 'چالش'), 1),
 (('975009', 'نوشتهٔ'), 1),
 (('975009', 'مواجه'), 1),
 (('975009', 'مردی'), 1),
 (('975009', 'شماره'), 1),
 (('975009', 'سنت'), 1),
 (('975009', 'زندگی'), 1),
 (('975009', 'زن'), 1),
 (('975009', 'زاویه'), 1),
 (('975009', 'روایت'), 1),
 (('975009', 'دید'), 1),
 (('975009', 'دچار'), 1),
 (('975009', 'دنیا'), 1),
 (('975009', 'داستان'), 1),
 ((

In [ ]:
doc_mat = ag_cache.map(lambda x : ((x[0][0]),(x[0][1],x[1]))).groupByKey().map(lambda x : (x[0],list(x[1])))
dm = doc_mat.collect()
dm

Out[27]: [('5905570',
  [('۱', 1),
   ('کشوری', 1),
   ('ورزشکار', 1),
   ('نقره', 1),
   ('میدانی', 1),
   ('مسکو', 1),
   ('مسابقات', 1),
   ('مدال', 1),
   ('مجموع', 1),
   ('شوروی', 1),
   ('سوسیالیستی', 1),
   ('زادهٔ', 1),
   ('دو', 1),
   ('جماهیر', 1),
   ('توان', 1),
   ('برندهٔ', 1),
   ('باشگاه', 1),
   ('بازی', 1),
   ('اهل', 1),
   ('الکساندر', 1),
   ('المللی', 1),
   ('اشاره', 1),
   ('اتحاد', 1)]),
 ('5254110',
  [('یک', 1),
   ('واقع', 1),
   ('نفر', 1),
   ('منطقه', 1),
   ('مسکونی', 1),
   ('لاتین', 1),
   ('شهرستان', 1),
   ('دام', 2),
   ('جمهوری', 1),
   ('جمعیت', 1),
   ('آق', 2),
   ('آذربایجان', 1)]),
 ('5199716',
  [('یک', 1),
   ('چاه', 1),
   ('واقع', 1),
   ('شهرستان', 1),
   ('روستا', 1),
   ('دهستان', 1),
   ('بیرجند', 1),
   ('بالا', 1),
   ('ایران', 1)]),
 ('2799604', [('یک', 1), ('گونه', 1), ('نام', 1), ('سرده', 1), ('درشت', 1)]),
 ('2787637',
  [('۱۹۷۵', 1),
   ('ژانر', 1),
   ('پرنده', 1),
   ('منتشر', 1),
   ('فیلمی', 1),
   ('سیاه', 1),
   ('جورج',

In [ ]:
doc_mat.count()


Out[28]: 9283

In [ ]:
len(lookup_table)

Out[35]: 5812

In [ ]:
import math
def calculate_idf(x, total_docs,lookup_table):
    dic = {}
    for k,v in x[1]:
#         look = lookup_table.filter(lambda x : x[0] == k).collect()
        for i in lookup_table:
            if i[0] == k:
                number_of_documents = i[1] # inja momkene to formula moshkel bashe
                dic[i[0]] = math.log(total_docs/number_of_documents) * v
                break
    return dic

# calculate_idf(dm[0], 100,lookup_table)
doc_mat.map(lambda x : calculate_idf(x,100, lookup_table)).collect()
# dm[0]

In [ ]:
form_2 = ag_cache.map(lambda x : (x[0][1],x[1])).reduceByKey(lambda x,y : x + y).map(lambda x : (x[1], x[0])).sortByKey(False)
lookup_table = form_2.collect()

In [ ]:
DOCS_LENGTH = doc_mat.count()
DOCS_LENGTH
WORDS_LENGTH = len(lookup_table)
tf_idf = np.zeros((DOCS_LENGTH, WORDS_LENGTH))

In [ ]:
# tf_idf = DOCS_LENGTH*[0]
# for i in tf_idf:
#     tf_idf[i] = WORDS_LENGTH * [0]

def find_index(word):
    index = 0
    for i in lt:
        if i[0] == word:
            return i, index
        index += 1

for i in range(DOCS_LENGTH):
    words = dm[i][1]
    for w in words:
        try:
            tup, index = find_index(w[0])
#         print(i, index, DOCS_LENGTH, tup[1], w[1])
        
            tf_idf[i][index] = math.log(DOCS_LENGTH/tup[1]) * w[1]
        except:
            pass
#         print(index)
#         print(tf_idf[i][index])
#         print(math.log(DOCS_LENGTH/tup[1]) * w[1])
# tf_idf
tf_idf

Out[76]: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
find_index("تاریخ")

In [ ]:
# 1. First caclulate each word freq across all doc
# 2. Join the all with ag_cache
# 3. groupByKey